In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jun  2 11:09:55 2018

@author: Marcelo
"""

import numpy as np
import numpy.ma as ma  # masked array
from machine_selection import set_screen_size_by_machine

# 2019-03-15
# https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
import pandas as pd
import csv

# 2019-11-06
import time

# 2019-11-03
from filebatch_class import (FileBatch)

# 2019-03-09
from teste_datas import (Spec)

# 2019-09-16
from gauss_funcs import gaus_fw, gaus_sig

# 2019-07-15
# https://docs.scipy.org/doc/numpy/reference/routines.polynomials.classes.html#basics
from numpy.polynomial import Polynomial as P
from numpy.polynomial.polynomial import (polyfit, polyval)

from scipy.signal import (cwt, ricker, find_peaks, find_peaks_cwt, peak_widths)
from scipy.ndimage import label, generate_binary_structure, find_objects  # 2019-09-18

from scipy.optimize import (curve_fit, root, lsq_linear)
from scipy.stats import (iqr)
from scipy.special import (expit)
import base_line_funcs as blf
import spectra_regions_funcs as spreg

# frm spectra_regions_funcs import (total_analysis2)
import csv
import pickle
from bokeh.models import ColumnDataSource, Whisker
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh import palettes

In [2]:
time.time()

1576328978.6097193

In [3]:
default_plot_width, default_plot_height = set_screen_size_by_machine()

In [4]:
# AQUI defino as larguras e quantas. Vamos detonar: tinha usado num=100
# 2019-10-01 Menos agora, usarei num=30

In [5]:
# widths = np.linspace(2,13,num=100)
# para espectros 8k
# widths = np.linspace(2,13,num=30)
# para espectros 4k
widths = np.linspace(2,7,num=30)
widths

array([2.        , 2.17241379, 2.34482759, 2.51724138, 2.68965517,
       2.86206897, 3.03448276, 3.20689655, 3.37931034, 3.55172414,
       3.72413793, 3.89655172, 4.06896552, 4.24137931, 4.4137931 ,
       4.5862069 , 4.75862069, 4.93103448, 5.10344828, 5.27586207,
       5.44827586, 5.62068966, 5.79310345, 5.96551724, 6.13793103,
       6.31034483, 6.48275862, 6.65517241, 6.82758621, 7.        ])

In [31]:
widths

array([2.        , 2.17241379, 2.34482759, 2.51724138, 2.68965517,
       2.86206897, 3.03448276, 3.20689655, 3.37931034, 3.55172414,
       3.72413793, 3.89655172, 4.06896552, 4.24137931, 4.4137931 ,
       4.5862069 , 4.75862069, 4.93103448, 5.10344828, 5.27586207,
       5.44827586, 5.62068966, 5.79310345, 5.96551724, 6.13793103,
       6.31034483, 6.48275862, 6.65517241, 6.82758621, 7.        ])

In [6]:
n_side_chans = 11
n_side_centr_ch = 5

In [7]:
nsca_det = 5

In [8]:
min_cwtpk = 13.0

In [9]:
# constante k: ver no Maxima: (sqrt(3)*6^(3/2))/(2^(3/2)*5^(5/4)*%pi^(1/4));
k = (np.sqrt(3)*6**(3/2))/(2**(3/2)*5**(5/4)*np.pi**(1/4))
k

0.9041542662259198

In [10]:
kfwhm = 2*np.sqrt(2*np.log(2))
kfwhm

2.3548200450309493

In [11]:
karea = np.sqrt(2*np.pi)
karea

2.5066282746310002

In [12]:
# 2019-09-26 Importar calibra´c~ao em efici^encia de arquivo CSV 

In [13]:
# 2019-09-26 Uma hora usarei isso: ler LibreOffice via Pandas!
# pd.read_excel('path_to_file.ods', engine='odf')

In [14]:
effics = pd.read_csv('2019-sep-25 ALMERA Samples CSV.csv',decimal=',',header=None)

In [15]:
effics

,0,1,2
0,46.54,0.008256,0.026447
1,475.35,0.002822,0.045887
2,563.23,0.002291,0.013179
3,569.32,0.002218,0.010851
4,604.70,0.002068,0.006972
5,795.84,0.001672,0.007033
6,801.93,0.001743,0.013259
7,1038.57,0.001614,0.067092
8,1167.94,0.001424,0.037835
9,1365.15,0.001292,0.026036


In [16]:
# 

In [17]:
npeffics = effics.to_numpy()

In [18]:
# ou entaum
npeffics = effics.to_numpy()
engys = npeffics[:,0]
effs = npeffics[:,1]
s_lneffs = npeffics[:,2]
lnengys = np.log(engys)
lneffs = np.log(effs)
lnengys

array([3.84031216, 6.16405137, 6.33368807, 6.34444267, 6.40473247,
       6.67939816, 6.68702132, 6.94560005, 7.06299679, 7.21901959,
       6.49475183, 6.72722811, 6.69799675, 7.06751591, 7.19481216,
       5.89847168, 4.39444915, 5.62184909, 5.71323763, 5.87498691,
       5.95025185, 4.08664836])

In [19]:
lneffs

array([-4.79681507, -5.87030943, -6.07876688, -6.11114939, -6.18117332,
       -6.39373476, -6.35214751, -6.42903971, -6.55428547, -6.65156387,
       -6.14508204, -6.24043886, -6.12659722, -6.59294454, -6.74648713,
       -5.63016033, -4.86627524, -5.69135995, -5.62128125, -5.68987957,
       -5.768321  , -4.85247836])

In [20]:
# lsq_linear: naum usarei por enquanto
# res = lsq_linear(effics, b, bounds=(lb, ub), lsmr_tol='auto', verbose=1)
# usarei 

In [21]:
# https://docs.scipy.org/doc/numpy/reference/routines.polynomials.classes.html

In [22]:
peff_w = polyfit(lnengys,lneffs,5,w=s_lneffs)
peff_w

array([-3.56316185e+02,  3.24741130e+02, -1.17786615e+02,  2.09925625e+01,
       -1.84188202e+00,  6.36705671e-02])

In [23]:
pfull = polyfit(lnengys,lneffs,5,full=True)
pfull

(array([ 1.29110608e+01, -2.11644869e+01,  9.55439800e+00, -2.03942161e+00,
         2.06411683e-01, -8.06276756e-03]),
 [array([0.11121854]),
  6,
  array([2.41600476e+00, 3.99024315e-01, 6.07435640e-02, 3.28130369e-03,
         1.80800009e-04, 3.56022063e-06]),
  4.884981308350689e-15])

In [24]:
pfull_w = polyfit(lnengys,lneffs,5,full=True,w=s_lneffs)
pfull_w

(array([-3.56316185e+02,  3.24741130e+02, -1.17786615e+02,  2.09925625e+01,
        -1.84188202e+00,  6.36705671e-02]),
 [array([8.04219408e-05]),
  6,
  array([2.42095475e+00, 3.70714692e-01, 3.92583140e-02, 2.73583863e-03,
         1.07477000e-04, 3.30990230e-06]),
  4.884981308350689e-15])

In [25]:
P(peff_w)

Polynomial([-3.56316185e+02,  3.24741130e+02, -1.17786615e+02,  2.09925625e+01,
       -1.84188202e+00,  6.36705671e-02], domain=[-1,  1], window=[-1,  1])

In [26]:
polyfit(lnengys,lneffs,3,w=s_lneffs)

array([12.23997754, -9.27243188,  1.65455979, -0.10164261])

In [28]:
# Bokeh graph w/ efficiencies
bokeffics = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5], title="effs",
   # y_range=[-1000.0, 1000.0], title="engys",
   x_axis_label='channels', y_axis_label='counts'
)

# base, lower, upper = [], [], []

base=lneffs
lower = lneffs - s_lneffs
upper = lneffs + s_lneffs

source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

# for i, eff in enumerate(list(effs)):
    # year_mpgs = df[df['yr'] == year]['mpg']
    # mpgs_mean = year_mpgs.mean()
    # mpgs_std = year_mpgs.std()
#    lower.append(lneff - mpgs_std)
#    upper.append(eff + mpgs_std)
#    base.append(year)

bokeffics.circle(lnengys, lneffs, legend_label="cwt peaks", color="navy",size=10)

# bokeffics.add_layout(
#     Whisker(source=source_error, base="base", upper="upper", lower="lower")
# )

# bokpksallscl.line(xs, y0s, color="lightgreen")
# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bokeffics)

In [29]:
# 2019-09-28 PAREY AKIH a parte do cahlculo de eficiehncias

In [30]:
# Alguns defs

In [30]:
##################################################

## 2019-nov-warp2.ipynb
## P.P.
## 2019-11-03: Integrando an'alise em lote de 2019_big-histogram.ipynb 
## com an'alise Cwt+ridges de 2019-cwt-analysis-real_with_effic-curve-Pandas.ipynb

fb = FileBatch()
spec = Spec()

In [31]:
fb.slotSetBatchCHN(
    '../Genie_Transfer/Si'
)

../Genie_Transfer/Si


In [32]:
# print (fb.arqslist)
print (fb.numarqstxt, ' arquivos')

Num de arquivos:       1098 


  arquivos


In [33]:
todos_si  = pd.Series(fb.lp)
todos_si

0               ../Genie_Transfer/Si/BGNOVO.Chn
1       ../Genie_Transfer/Si/si2016/SI14816.Chn
2       ../Genie_Transfer/Si/si2016/SI08216.Chn
3       ../Genie_Transfer/Si/si2016/SI14416.Chn
4       ../Genie_Transfer/Si/si2016/SI09416.Chn
                         ...                   
1093    ../Genie_Transfer/Si/Si2006/si12606.Chn
1094    ../Genie_Transfer/Si/Si2006/SI12206.Chn
1095    ../Genie_Transfer/Si/Si2006/SI12306.Chn
1096    ../Genie_Transfer/Si/Si2006/SI07906.Chn
1097    ../Genie_Transfer/Si/Si2006/si12906.Chn
Length: 1098, dtype: object

In [34]:
len(fb.lp)

1098

In [35]:
fb.slotSetBatchCHN(
    '../Genie_Transfer/Si/SI2019'
)
print (fb.numarqstxt, ' arquivos')
todos_si2019  = pd.Series(fb.lp)
todos_si2019

../Genie_Transfer/Si/SI2019
Num de arquivos:         28 


  arquivos


0     ../Genie_Transfer/Si/SI2019/SI5-19-2.Chn
1      ../Genie_Transfer/Si/SI2019/SI02819.Chn
2      ../Genie_Transfer/Si/SI2019/SI04019.Chn
3      ../Genie_Transfer/Si/SI2019/SI00119.Chn
4      ../Genie_Transfer/Si/SI2019/SI03119.Chn
5      ../Genie_Transfer/Si/SI2019/SI04319.Chn
6      ../Genie_Transfer/Si/SI2019/SI04219.Chn
7      ../Genie_Transfer/Si/SI2019/si00919.Chn
8      ../Genie_Transfer/Si/SI2019/si00719.Chn
9      ../Genie_Transfer/Si/SI2019/SI01419.Chn
10     ../Genie_Transfer/Si/SI2019/SI04119.Chn
11     ../Genie_Transfer/Si/SI2019/SI03919.Chn
12     ../Genie_Transfer/Si/SI2019/SI00219.Chn
13     ../Genie_Transfer/Si/SI2019/SI01519.Chn
14     ../Genie_Transfer/Si/SI2019/SI01619.Chn
15     ../Genie_Transfer/Si/SI2019/SI03819.Chn
16     ../Genie_Transfer/Si/SI2019/si00619.Chn
17     ../Genie_Transfer/Si/SI2019/SI02419.Chn
18     ../Genie_Transfer/Si/SI2019/SI02619.Chn
19     ../Genie_Transfer/Si/SI2019/SI03219.Chn
20     ../Genie_Transfer/Si/SI2019/si00319.Chn
21     ../Gen

In [36]:
# 2019-11-04 Para cada fn2:
# Modificado em 2019-11-23

def analisa_um_fn(fn2):
    spec2 = Spec()
    spec2.readSp( fn2 )
    # spec2.spLngData = 4096
    xs = np.linspace(0,spec2.spLngData-1,spec2.spLngData)
    y0s = np.asarray(spec2.spCounts)

    ipks2 = find_peaks_cwt(y0s,widths=[widths[1]],wavelet=ricker,max_distances=widths/4,min_snr=1.0,noise_perc=10)
    ipks2

    pk_per_scal = []
    for wid in widths:
        pk_per_scal.append(find_peaks_cwt
                           (y0s,widths=[wid],wavelet=ricker,max_distances=widths/4,min_snr=1.0,noise_perc=10))

    pks_cwt_flatten = []
    for isc, pksc in enumerate(pk_per_scal):
        for pk in pksc:
            pks_cwt_flatten.append([pk,widths[isc]])
    len(pks_cwt_flatten)
    
    pks_cwt_flatten = []
    for isc, pksc in enumerate(pk_per_scal):
        for pk in pksc:
            pks_cwt_flatten.append([pk,widths[isc]])
    
    ar1 = np.asarray(pks_cwt_flatten)
    
    # imag1: imagem de pixels binahrios dos picos nas escalas
    imag1 = np.zeros((len(widths),len(xs)))
    for isc, pksc in enumerate(pk_per_scal):
        for pk in pksc:
            imag1[isc,pk] = 1
    stru_diag_also = generate_binary_structure(2,2)
    lblimag1, num_feat_imag1 = label(imag1,stru_diag_also)
    ridges = find_objects(lblimag1)
    ridges_rectangles = [lblimag1[loc] for loc in ridges]
    cwtmatr = cwt(y0s, ricker, widths)

    xpk = []
    ypk = []
    centrpk = []
    cwtpk = []
    nscpk = []
    regions_slcs = []
    for iridge, ridge in enumerate(ridges):
        lbl_ridge = iridge+1
        nscpk.append( ridge[0].stop - ridge[0].start)
        labeled = lblimag1[ridge]
        cwts_ridge = cwtmatr[ridge]
        retang3 = ma.masked_where(labeled!=lbl_ridge,cwts_ridge)
        maxim3 = np.max(retang3)
        flat_retang_ind = np.argmax(retang3)
        retang_indices = np.unravel_index(flat_retang_ind,np.shape(retang3))
        # print(iridge)
        # print(ridge)
        # print(maxim3)
        # print(retang_indices)
        indices_max_in_ridge = tuple(map(lambda x, y: x + y, retang_indices, (ridge[0].start, ridge[1].start)))
        cwtpk.append(maxim3)
        xpk.append( indices_max_in_ridge[1] )
        ypk.append( widths[indices_max_in_ridge[0]] )
        region_slc = slice(xpk[-1]-n_side_chans, xpk[-1]+n_side_chans+1)
        regions_slcs.append(region_slc)
        # 2019-10-02 C'alculo de centroide
        region_centr = slice(xpk[-1]-n_side_centr_ch, xpk[-1]+n_side_centr_ch+1)
        centrpk.append( np.sum(xs[region_centr]*y0s[region_centr]) / np.sum(y0s[region_centr]))

    nscpk_ma = ma.masked_less(nscpk, nsca_det)
    cwtpk_ma = ma.masked_less(cwtpk, min_cwtpk)
    valid_peaks = ~(cwtpk_ma.mask | nscpk_ma.mask)
    regions_slcs_valid = np.asarray(regions_slcs)[valid_peaks]

    ypkregs = []
    fwhms = []
    gau_areas = []
    unc_gau_areas = []
    peak_constructed_spectrum = np.zeros(len(xs))
    for ipk, pk in enumerate(valid_peaks):
        if pk:
            # ypicoregio = gaus_sig(xs[regions_slcs[ipk2]],sigmamaximo,gau_hei,centroide)ypicoregio
            # subtrair o pica v'alido
            escalmaxima = ypk[ipk]
            maxcwt = cwtpk[ipk]
            centroide = centrpk[ipk]
            sigmamaximo = escalmaxima / np.sqrt(5)
            fwhm = kfwhm * sigmamaximo
            gau_hei = k * maxcwt / np.sqrt(sigmamaximo)
            gau_area = gau_hei * sigmamaximo * karea
            # print(escalmaxima, maxcwt, centroide, sigmamaximo, gau_hei)
            region_start = regions_slcs[ipk].start
            # print(region_start)
            ypicoregio = gaus_sig(xs[regions_slcs[ipk]],sigmamaximo,gau_hei,centroide)
            # print(ypicoregio)
            ypkregs.append(ypicoregio)
            fwhms.append(fwhm)
            gau_areas.append(gau_area)
            peak_constructed_spectrum[regions_slcs[ipk]] += ypicoregio
    # peak_constructed_spectrum
    baseline_spectrum = y0s - peak_constructed_spectrum
    len_bl = len(baseline_spectrum)
    bl_median_smoother = np.zeros(len_bl)
    bl_median_smoother[20:len_bl-20] = np.asarray(
        [np.median(baseline_spectrum[i-n_side_chans:i+n_side_chans+1]) for i in range(20,len_bl-20) ])
    
    for ipk,regio in enumerate(regions_slcs_valid):
        bline_cts = np.sum(bl_median_smoother[regio])
        unc_gau_area = np.sqrt(gau_areas[ipk]+bline_cts)
        # print ('gau_areas: ', gau_areas[ipk])
        # print ('unc_gau_area: ', unc_gau_area)
        # print ('bline_cts: ', bline_cts)
        # print ('------------------------')
        unc_gau_areas.append(unc_gau_area)
    y0s_xpk_valid = y0s[xpk][valid_peaks]
    y0s_xpk_invld = y0s[xpk][~valid_peaks]
    centrpk_valid = np.asarray(centrpk)[valid_peaks]
    centrpk_invld = np.asarray(centrpk)[~valid_peaks]
    # df2 = pd.DataFrame({'Centr': centrpk_valid,
    #                     'Fwhm': fwhms,
    #                     'Area': gau_areas,
    #                     'SArea': unc_gau_areas})
    return spec2, ar1, xpk, ypk, peak_constructed_spectrum, bl_median_smoother, y0s_xpk_valid, y0s_xpk_invld, \
    valid_peaks, regions_slcs_valid, centrpk_valid, centrpk_invld, fwhms, gau_areas, unc_gau_areas

In [37]:
# todos_si = todos_si2019

inames = []
fnames = []
ltimes = []
statimes = []
for iname, fname in enumerate(todos_si):
    inames.append(iname)
    fnames.append(fname)
    
    spec2 = Spec()
    if spec2.readSp( fname ) >= 0:
        statimes.append(spec2.diahora)
        ltimes.append(spec2.spLVTime)
    
dfraA = pd.DataFrame({'IdSpec': inames,
                    'FName': fnames,
                    'StTime': statimes,
                    'LvTime': ltimes})
dfraA

,IdSpec,FName,StTime,LvTime
0,0,../Genie_Transfer/Si/BGNOVO.Chn,2013-12-13 15:27:02,237389.66
1,1,../Genie_Transfer/Si/si2016/SI14816.Chn,2016-12-21 15:33:47,50000.00
2,2,../Genie_Transfer/Si/si2016/SI08216.Chn,2016-06-29 16:29:37,50000.00
3,3,../Genie_Transfer/Si/si2016/SI14416.Chn,2016-12-08 19:32:14,50000.00
4,4,../Genie_Transfer/Si/si2016/SI09416.Chn,2016-07-26 15:21:10,50000.00
...,...,...,...,...
1093,1093,../Genie_Transfer/Si/Si2006/si12606.Chn,2006-09-25 16:19:09,50000.00
1094,1094,../Genie_Transfer/Si/Si2006/SI12206.Chn,2006-09-04 16:19:13,50000.00
1095,1095,../Genie_Transfer/Si/Si2006/SI12306.Chn,2006-09-08 01:04:12,50000.00
1096,1096,../Genie_Transfer/Si/Si2006/SI07906.Chn,2006-06-05 15:26:05,50000.00


In [38]:
dfraA.to_pickle('todos_si_specs.pkl')

In [39]:
# https://stackoverflow.com/questions/17098654/how-to-store-a-dataframe-using-pandas

# dfraA.to_pickle('todos_si2019_specs.pkl')  # where to save it, usually as a .pkl
# dfraA2 = pd.read_pickle('dfraA.pkl')

In [40]:
dfraA.tail(6)

,IdSpec,FName,StTime,LvTime
1092,1092,../Genie_Transfer/Si/Si2006/si15006r.Chn,2006-11-06 08:49:03,1000.0
1093,1093,../Genie_Transfer/Si/Si2006/si12606.Chn,2006-09-25 16:19:09,50000.0
1094,1094,../Genie_Transfer/Si/Si2006/SI12206.Chn,2006-09-04 16:19:13,50000.0
1095,1095,../Genie_Transfer/Si/Si2006/SI12306.Chn,2006-09-08 01:04:12,50000.0
1096,1096,../Genie_Transfer/Si/Si2006/SI07906.Chn,2006-06-05 15:26:05,50000.0
1097,1097,../Genie_Transfer/Si/Si2006/si12906.Chn,2006-10-02 17:16:05,50000.0


In [41]:
umaanalise = dfraA[13:14]

In [42]:
umaanalise

,IdSpec,FName,StTime,LvTime
13,13,../Genie_Transfer/Si/si2016/SI14216.Chn,2016-12-06 18:19:07,50000.0


In [43]:
allpeaks_df = pd.DataFrame()
allpeaks_df

""


In [44]:
# len(dfraA2)

In [45]:
df2s = pd.DataFrame()

In [46]:
df2s

""


In [47]:
todos_si

0               ../Genie_Transfer/Si/BGNOVO.Chn
1       ../Genie_Transfer/Si/si2016/SI14816.Chn
2       ../Genie_Transfer/Si/si2016/SI08216.Chn
3       ../Genie_Transfer/Si/si2016/SI14416.Chn
4       ../Genie_Transfer/Si/si2016/SI09416.Chn
                         ...                   
1093    ../Genie_Transfer/Si/Si2006/si12606.Chn
1094    ../Genie_Transfer/Si/Si2006/SI12206.Chn
1095    ../Genie_Transfer/Si/Si2006/SI12306.Chn
1096    ../Genie_Transfer/Si/Si2006/SI07906.Chn
1097    ../Genie_Transfer/Si/Si2006/si12906.Chn
Length: 1098, dtype: object

In [48]:
# Loop para todos si
# 2019-11-23 Mudar a criacao do dataframe como estah aqui:
t1 = time.time()
print(t1)
# peaks2019_lst = list()
peaks_lst = list()

for iname, fname in enumerate(todos_si):
    print(iname)
    print(fname)
    
    spec2, ar1, xpk, ypk, peak_constructed_spectrum, bl_median_smoother, y0s_xpk_valid, y0s_xpk_invld, \
    valid_peaks, regions_slcs_valid, centrpk_valid, centrpk_invld, fwhms, gau_areas, unc_gau_areas \
    = analisa_um_fn(fname)
    
    peaks_lst.append([iname,centrpk_valid,fwhms,gau_areas, unc_gau_areas])
    # peaks2019_lst.append([iname,centrpk_valid,fwhms,gau_areas, unc_gau_areas])

    # df2['IdSpec']=iname
    # print(df2)
    # df2s = df2s.append(df2)
# peaks2019_lst
peaks_lst

# df2s
t2 = time.time()
print(t2)
telapsed = t2-t1
print(telapsed)

1574601196.0872524
0
../Genie_Transfer/Si/BGNOVO.Chn
1
../Genie_Transfer/Si/si2016/SI14816.Chn
2
../Genie_Transfer/Si/si2016/SI08216.Chn
3
../Genie_Transfer/Si/si2016/SI14416.Chn
4
../Genie_Transfer/Si/si2016/SI09416.Chn
5
../Genie_Transfer/Si/si2016/SI09116.Chn
6
../Genie_Transfer/Si/si2016/SI01516.Chn
7
../Genie_Transfer/Si/si2016/SI02816.Chn
8
../Genie_Transfer/Si/si2016/SI13316.Chn
9
../Genie_Transfer/Si/si2016/SI14516.Chn
10
../Genie_Transfer/Si/si2016/SI07416.Chn
11
../Genie_Transfer/Si/si2016/SI08816.Chn
12
../Genie_Transfer/Si/si2016/si13916.Chn
13
../Genie_Transfer/Si/si2016/SI14216.Chn
14
../Genie_Transfer/Si/si2016/SI14316.Chn
15
../Genie_Transfer/Si/si2016/SI13016.Chn
16
../Genie_Transfer/Si/si2016/SI02016.Chn
17
../Genie_Transfer/Si/si2016/SI10916.Chn
18
../Genie_Transfer/Si/si2016/SI15116.Chn
19
../Genie_Transfer/Si/si2016/SI13716.Chn
20
../Genie_Transfer/Si/si2016/SI02716.Chn
21
../Genie_Transfer/Si/si2016/SI04516.Chn
22
../Genie_Transfer/Si/si2016/SI6116.Chn
23
../Genie

/home/marcelo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in double_scalars


30
../Genie_Transfer/Si/si2016/SI11016.Chn
31
../Genie_Transfer/Si/si2016/SI13616.Chn
32
../Genie_Transfer/Si/si2016/SI02916.Chn
33
../Genie_Transfer/Si/si2016/SI10316.Chn
34
../Genie_Transfer/Si/si2016/SI06716.Chn
35
../Genie_Transfer/Si/si2016/SI11916.Chn
36
../Genie_Transfer/Si/si2016/SI04916.Chn
37
../Genie_Transfer/Si/si2016/SI2616.Chn
38
../Genie_Transfer/Si/si2016/SI12516.Chn
39
../Genie_Transfer/Si/si2016/SI04616.Chn
40
../Genie_Transfer/Si/si2016/SI14716.Chn
41
../Genie_Transfer/Si/si2016/SI11316.Chn
42
../Genie_Transfer/Si/si2016/SI08116.Chn
43
../Genie_Transfer/Si/si2016/BG DET E.Chn
44
../Genie_Transfer/Si/si2016/SI05016.Chn
45
../Genie_Transfer/Si/si2016/SI00716.Chn
46
../Genie_Transfer/Si/si2016/SI14016.Chn
47
../Genie_Transfer/Si/si2016/SI11816.Chn
48
../Genie_Transfer/Si/si2016/SI07716.Chn
49
../Genie_Transfer/Si/si2016/SI01916.Chn
50
../Genie_Transfer/Si/si2016/SI08516.Chn
51
../Genie_Transfer/Si/si2016/SI14116.Chn
52
../Genie_Transfer/Si/si2016/SI06416.Chn
53
../Genie

/home/marcelo/anaconda3/lib/python3.7/site-packages/scipy/signal/_peak_finding.py:1185: RuntimeWarning: divide by zero encountered in double_scalars
  snr = abs(cwt[line[0][0], line[1][0]] / noises[line[1][0]])


533
../Genie_Transfer/Si/si2015/SI02615.Chn
534
../Genie_Transfer/Si/si2015/Lavagem 3.Chn
535
../Genie_Transfer/Si/si2015/si01015.Chn
536
../Genie_Transfer/Si/si2015/SI07015.Chn
537
../Genie_Transfer/Si/si2015/SI09815.Chn
538
../Genie_Transfer/Si/si2015/SI05315.Chn
539
../Genie_Transfer/Si/si2015/SI03215.Chn
540
../Genie_Transfer/Si/si2015/SI3015.Chn
541
../Genie_Transfer/Si/si2015/SI10615.Chn
542
../Genie_Transfer/Si/si2015/SI5115.Chn
543
../Genie_Transfer/Si/si2015/SI04115.Chn
544
../Genie_Transfer/Si/si2015/SI09515.Chn
545
../Genie_Transfer/Si/si2015/SI12015.Chn
546
../Genie_Transfer/Si/si2015/SI09115.Chn
547
../Genie_Transfer/Si/si2015/SI08915.Chn
548
../Genie_Transfer/Si/si2015/SI04215.Chn
549
../Genie_Transfer/Si/si2015/Lavagem 1.Chn
550
../Genie_Transfer/Si/si2015/SI08415.Chn
551
../Genie_Transfer/Si/si2015/SI9715.Chn
552
../Genie_Transfer/Si/si2015/SI05115.Chn
553
../Genie_Transfer/Si/si2015/SI01515.Chn
554
../Genie_Transfer/Si/si2015/Lavagem 2.Chn
555
../Genie_Transfer/Si/si20

In [49]:
aux_list=[]
saco=[]
for i in peaks_lst:
    rolao = np.asarray(i[1:5]).transpose().tolist()
    # print (rolao)
    for j in rolao:
        linha=[]
        linha.append(i[0])
        [linha.append(k) for k in j]
        saco.append(linha)

In [50]:
print(t1)
print(t2)
print(telapsed)

1574601196.0872524
1574606219.6482701
5023.561017751694


In [51]:
df2s

""


In [52]:
np.shape(saco)

(59050, 5)

In [53]:
dfrapk = pd.DataFrame(saco, columns=list(['IdSpec_2','Centr','Fwhm','Area','SArea']))
dfrapk

,IdSpec_2,Centr,Fwhm,Area,SArea
0,0,40.553747,4.648199,3815.384906,103.438491
1,0,46.086864,4.285058,2682.249310,124.655743
2,0,94.958163,3.740347,3441.258574,171.046572
3,0,106.967747,2.469356,1512.384738,162.374074
4,0,115.714642,2.106215,2142.793388,159.754004
...,...,...,...,...,...
59045,1097,2208.983539,5.556050,117.731428,62.049194
59046,1097,2314.983583,6.645472,140.957080,65.033515
59047,1097,2273.959508,4.829769,60.258412,63.392232
59048,1097,2854.884221,7.371753,327.031536,76.470880


In [55]:
dfrapk.to_pickle('todos_si_peaks.pkl')
# dfrapk.to_pickle('todos_si2019_peaks.pkl') 

In [58]:
### 2019-11-24
### Encerrar este programa aqui.
### O que estah abaixo eh mais antigo e especihfico. Transportar para outro cohdigo.

In [56]:
# Vari'aveis importantes definidas at'e aqui

# n_side_chans = 11
# n_side_centr_ch = 3

# nscpk - n'umero de escalas (=pontos) que comp~oe o ridge 
# xpk - canais dos picos ap'os an'alise dos ridges, com coeficiente cwt m'ahximo
# ypk - escalas dos picos ap'os an'alise dos ridges, com coeficiente cwt m'ahximo
# cwtpk - coeficiente cwt correspondente a (xpk, ypk)
# centrpk - centroides dos xpk
# regions_slcs - slices das regi~oes dos picos

In [57]:
# 2019-10-02 Selecionando picos vahlidos pelo critehrio seguinte:
# nsca_det: nuhmero de escalas sucessivas em que eh detectado (= nuhmero de pontos do ridge)
# Supondo que naum haja bifurca,caum, eh a altura do retahngulo que o contehm.
# Adotarei 5 por enquanto
# ou 7

In [158]:
# 2019-10-02 Parei aqui.
# Agora, montar um array com todos os picos ajustados; e subtrair de xs.
# Plotar para ver como fica. Deve ficar uma linha base com alguns picos n~ao subtra'idos.
# Prosseguindo:

In [159]:
# Vari'aveis importantes definidas at'e aqui

# xs - canais
# y0s - contagens
# widhts - escalas

# pk_per_scal: lista de arrays; cada array corresponde a uma escala e contehm os canais dos picos detectados na escala.
# pks_cwt_flatten: lista contihnua em que cada elemento eh uma lista de 2 elementos: [pk, width].

# nscpk - n'umero de escalas (=pontos) que comp~oe o ridge 
# xpk - canais dos picos ap'os an'alise dos ridges, com coeficiente cwt m'ahximo
# ypk - escalas dos picos ap'os an'alise dos ridges, com coeficiente cwt m'ahximo
# cwtpk - coeficiente cwt correspondente a (xpk, ypk)
# centrpk - centroides dos xpk
# regions_slcs - slices das regi~oes dos picos
# nsca_det: nuhmero m'inimo de escalas sucessivas em que eh detectado (= nuhmero de pontos do ridge)
# nscpk_ma - nscpk com m'ascara para valores < nsca_det 

In [160]:
fn2 = '..\Genie_Transfer\Si\SI2019\SI04219.Chn'
spec2, ar1, df2, xpk, ypk, peak_constructed_spectrum, bl_median_smoother, y0s_xpk_valid, y0s_xpk_invld, valid_peaks, \
regions_slcs_valid, centrpk_valid, centrpk_invld, fwhms = analisa_um_fn(fn2)

Erro de arquivo CHN!!!


AttributeError: 'Spec' object has no attribute 'spLngData'

In [161]:
spec2.diahora

datetime.datetime(2019, 5, 29, 19, 34, 13)

In [162]:
spec2.spRLTime

50145.3

In [163]:
if spec2.readSp( fn2 ) >= 0:
    print (spec2.spLngData)

Erro de arquivo CHN!!!


In [164]:
xs = np.linspace(0,spec2.spLngData-1,spec2.spLngData)
y0s = np.asarray(spec2.spCounts)

In [165]:
bokspec = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   y_axis_type="log",
   y_range=[10**1, 10**5], title=fn2,
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
bokspec.circle(xs, y0s, legend="counts", color="green",size=2)
# bokspec.line(xs, y0s, color="green")
bokspec.line(xs, peak_constructed_spectrum, color="blue")
# bokspec.line(xs, baseline_spectrum, color="orange")
bokspec.line(xs, bl_median_smoother, color="magenta")
bokspec.cross(centrpk_valid, y0s_xpk_valid, color="red",size=16)
bokspec.cross(centrpk_invld, y0s_xpk_invld, color="magenta",size=8)
# fwhms:
bokspec.circle(centrpk_valid, np.asarray(fwhms)*2000, color="magenta",size=8)
show(bokspec)

In [166]:
# Bokeh graph w/ peaks in all scales
# 2019-10-01 Now including the cwt maximum in each ridge
bokpksallscl = figure(
   plot_width =default_plot_width,
   # plot_height=default_plot_height,
   plot_height=500,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5], title="synth",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
bokpksallscl.circle(ar1[:,0], ar1[:,1], legend="cwt peaks", color="navy",size=2)
# bokpksallscl.line(xs, y0s, color="lightgreen")
bokpksallscl.cross(xpk, ypk, color="red",size=12)
show(bokpksallscl)

In [167]:
# 2019-11-04 Parei aqui.

In [168]:
len(regions_slcs_valid)

41

In [169]:
len(centrpk_valid)

41

In [171]:
# np.sum(bl_median_smoother[regions_slcs[ipk]])
regions_slcs_valid

array([slice(45, 68, None), slice(96, 119, None), slice(108, 131, None),
       slice(117, 140, None), slice(220, 243, None),
       slice(278, 301, None), slice(317, 340, None),
       slice(357, 380, None), slice(368, 391, None),
       slice(404, 427, None), slice(450, 473, None),
       slice(495, 518, None), slice(509, 532, None),
       slice(580, 603, None), slice(621, 644, None),
       slice(742, 765, None), slice(765, 788, None),
       slice(775, 798, None), slice(797, 820, None),
       slice(860, 883, None), slice(912, 935, None),
       slice(939, 962, None), slice(994, 1017, None),
       slice(1052, 1075, None), slice(1114, 1137, None),
       slice(1130, 1153, None), slice(1249, 1272, None),
       slice(1313, 1336, None), slice(1490, 1513, None),
       slice(1547, 1570, None), slice(1632, 1655, None),
       slice(1648, 1671, None), slice(1681, 1704, None),
       slice(1744, 1767, None), slice(1780, 1803, None),
       slice(1968, 1991, None), slice(2344, 2367, None

In [172]:
len(regions_slcs_valid)

41

In [173]:
valid_peaks

array([ True, False,  True,  True,  True, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,